In [1]:
!pip install fastapi uvicorn nest_asyncio

import nest_asyncio
nest_asyncio.apply()

from fastapi import FastAPI, UploadFile, File
from pydantic import BaseModel
import random
import asyncio
import threading
from uvicorn import Config, Server

app = FastAPI()

class TextInput(BaseModel):
    text: str

class EmotionInput(BaseModel):
    emotion: str

music_db = {
    "happy": [
        {"name": "Happy", "artist": "Pharrell Williams", "link": "https://open.spotify.com/track/60nZcImufyMA1MKQY3dcCH"},
        {"name": "Best Day of My Life", "artist": "American Authors", "link": "https://open.spotify.com/track/6Ff53KvcvAj5U7Z0C0b6uB"}
    ],
    "sad": [
        {"name": "Someone Like You", "artist": "Adele", "link": "https://open.spotify.com/track/4kflIGfjdZJW4ot2ioixTB"},
        {"name": "Fix You", "artist": "Coldplay", "link": "https://open.spotify.com/track/7LVHVU3tWfcxj5aiPFEW4Q"}
    ],
    "neutral": [
        {"name": "Let It Be", "artist": "The Beatles", "link": "https://open.spotify.com/track/7iN1s7xHE4ifF5povM6A48"}
    ]
}

@app.post("/predict/face")
async def predict_face(file: UploadFile = File(...)):
    emotion = random.choice(["happy", "sad", "neutral"])
    return {"emotion": emotion}

@app.post("/predict/text")
async def predict_text(input: TextInput):
    text = input.text.lower()
    if any(word in text for word in ["happy", "joy", "great"]):
        emotion = "happy"
    elif any(word in text for word in ["sad", "down", "cry"]):
        emotion = "sad"
    else:
        emotion = "neutral"
    return {"emotion": emotion}

@app.post("/recommend")
async def recommend_music(data: EmotionInput):
    emotion = data.emotion.lower()
    return music_db.get(emotion, music_db["neutral"])

# Function to run FastAPI in background
def run_server():
    config = Config(app=app, host="0.0.0.0", port=8888, log_level="info")
    server = Server(config=config)
    asyncio.run(server.serve())

threading.Thread(target=run_server, daemon=True).start()


Defaulting to user installation because normal site-packages is not writeable


INFO:     Started server process [19248]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8888 (Press CTRL+C to quit)


In [2]:
!pip install fastapi uvicorn nest_asyncio requests

import nest_asyncio, threading, asyncio, time
from fastapi import FastAPI, UploadFile, File
from pydantic import BaseModel
import random
from uvicorn import Config, Server
import requests

nest_asyncio.apply()

app = FastAPI()

class TextInput(BaseModel):
    text: str

class EmotionInput(BaseModel):
    emotion: str

music_db = {
    "happy": [
        {"name": "Happy", "artist": "Pharrell Williams", "link": "https://open.spotify.com/track/60nZcImufyMA1MKQY3dcCH"},
        {"name": "Best Day of My Life", "artist": "American Authors", "link": "https://open.spotify.com/track/6Ff53KvcvAj5U7Z0C0b6uB"}
    ],
    "sad": [
        {"name": "Someone Like You", "artist": "Adele", "link": "https://open.spotify.com/track/4kflIGfjdZJW4ot2ioixTB"},
        {"name": "Fix You", "artist": "Coldplay", "link": "https://open.spotify.com/track/7LVHVU3tWfcxj5aiPFEW4Q"}
    ],
    "neutral": [
        {"name": "Let It Be", "artist": "The Beatles", "link": "https://open.spotify.com/track/7iN1s7xHE4ifF5povM6A48"}
    ]
}

@app.post("/predict/face")
async def predict_face(file: UploadFile = File(...)):
    emotion = random.choice(["happy", "sad", "neutral"])
    return {"emotion": emotion}

@app.post("/predict/text")
async def predict_text(input: TextInput):
    text = input.text.lower()
    if any(word in text for word in ["happy", "joy", "great"]):
        emotion = "happy"
    elif any(word in text for word in ["sad", "down", "cry"]):
        emotion = "sad"
    else:
        emotion = "neutral"
    return {"emotion": emotion}

@app.post("/recommend")
async def recommend_music(data: EmotionInput):
    emotion = data.emotion.lower()
    return music_db.get(emotion, music_db["neutral"])

@app.get("/test")
async def test():
    return {"status": "Server is running"}

# Run FastAPI in background thread
def run_server():
    config = Config(app=app, host="0.0.0.0", port=8889, log_level="info")
    server = Server(config=config)
    asyncio.run(server.serve())


threading.Thread(target=run_server, daemon=True).start()

# Wait a moment for server to start
time.sleep(2)

# --- Testing endpoints safely ---
base_url = "http://127.0.0.1:8889"

# Test /test endpoint
print("Test endpoint:", requests.get(f"{base_url}/test").json())

# Test /predict/text
print("Text emotion:", requests.post(f"{base_url}/predict/text", json={"text": "I am feeling great"}).json())

# Test /recommend
print("Music recommendation:", requests.post(f"{base_url}/recommend", json={"emotion": "happy"}).json())


Defaulting to user installation because normal site-packages is not writeable


INFO:     Started server process [19248]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8889 (Press CTRL+C to quit)


INFO:     127.0.0.1:60172 - "GET /test HTTP/1.1" 200 OK
Test endpoint: {'status': 'Server is running'}
INFO:     127.0.0.1:60173 - "POST /predict/text HTTP/1.1" 200 OK
Text emotion: {'emotion': 'happy'}
INFO:     127.0.0.1:60174 - "POST /recommend HTTP/1.1" 200 OK
Music recommendation: [{'name': 'Happy', 'artist': 'Pharrell Williams', 'link': 'https://open.spotify.com/track/60nZcImufyMA1MKQY3dcCH'}, {'name': 'Best Day of My Life', 'artist': 'American Authors', 'link': 'https://open.spotify.com/track/6Ff53KvcvAj5U7Z0C0b6uB'}]


In [8]:
import requests

# Define the port your FastAPI server is running on
PORT = 8888  # replace with the actual port if different
base_url = f"http://127.0.0.1:{PORT}"

# Path to your local image file
image_path = "WIN_20251014_21_42_07_Pro.jpg"  # make sure this file exists in the same folder

# Send the image to the /predict/face endpoint
try:
    with open(image_path, "rb") as f:
        response = requests.post(
            f"{base_url}/predict/face",
            files={"file": (image_path, f, "image/jpeg")}
        )
    # Check if the server responded with JSON
    try:
        result = response.json()
        print("Face emotion prediction:", result)
    except requests.JSONDecodeError:
        print("Server did not return valid JSON. Response text:", response.text)
except FileNotFoundError:
    print(f"File not found: {image_path}. Make sure the file exists.")
except requests.exceptions.RequestException as e:
    print("Request failed:", e)

Server did not return valid JSON. Response text: <!DOCTYPE HTML>
<html>

<head>

    <meta charset="utf-8">

    <title>Jupyter Server</title>
    <link id="favicon" rel="shortcut icon" type="image/x-icon" href="/static/favicon.ico?v=50afa725b5de8b00030139d09b38620224d4e7dba47c07ef0e86d4643f30c9bfe6bb7e1a4a1c561aa32834480909a4b6fe7cd1e17f7159330b6b5914bf45a880">
    
    <link rel="stylesheet" href="/static/style/bootstrap.min.css?v=0e8a7fbd6de23ad6b27ab95802a0a0915af6693af612bc304d83af445529ce5d95842309ca3405d10f538d45c8a3a261b8cff78b4bd512dd9effb4109a71d0ab" />
    <link rel="stylesheet" href="/static/style/bootstrap-theme.min.css?v=8b2f045cb5b4d5ad346f6e816aa2566829a4f5f2783ec31d80d46a57de8ac0c3d21fe6e53bcd8e1f38ac17fcd06d12088bc9b43e23b5d1da52d10c6b717b22b3" />
    <link rel="stylesheet" href="/static/style/index.css?v=30372e3246a801d662cf9e3f9dd656fa192eebde9054a2282449fe43919de9f0ee9b745d7eb49d3b0a5e56357912cc7d776390eddcab9dac85b77bdb17b4bdae" />
    <meta http-equiv="X-UA-Compa